<big><b><i>Parsing Script: 2021 brainSPORT rsfMRI TBI Literature Review</i></b></big>


<b>README</b>: This script was written July 14th, 2021 by Daniel Frees, an undergraduate student of UCLA for use by a research team at the Steve Tisch brainSPORT Lab. If you wish to reuse some or all of this code for other purposes email me: danielfrees@g.ucla.edu

Input directories for the CSV files containing your meta-analysis info. All of the necessary edits can be performed in the Directory class, by editing the baseDir (base directory) and the additional specific locations of each network file. 

<b>Functions: </b>

----
cleanData(datalist): cleanData() takes in a list of pandas dataframes (these are generated in my code shortly below the directory inputs), and outputs clean dataframes for the columns relevant to our analysis. I used natural language processing (NLP) to determine what our final verdict for each result/classification was, and then edited the output list of dataframes accordingly. 

•Rules: the final determined Result must be written leftmost in its cell so the correct keyword can be identified. Results and Chronicities must be listed for each finding, but other classifications will autofill down wherever they have not been filled in (as we typically only wrote these out for the first finding of a given paper). Certain mispellings and miscapitalizations are allowed, but I did not write a full Trie or other spellchecker, so try to avoid typos as much as possible. I made sure through multiple iterations that this was catching all of our most common mispellings/ miscaps.

-----
runStats(datalist, stats): runStats() takes in a list of pandas dataframes (these should be the cleaned ones output from cleanData()) as well as a 2D 'stats' list (a list of size 2 lists, where each size 2 list is a pair: a classification, and a count of how many times that classification shows up in the data (this should start at 0 for all classifications the same way I initially set up my stats object). It outputs a modified 2D 'stats' list with the counts added up for each classification after parsing through the provided datalist.

•Rules: Mostly just be careful to pass a stats 2D list which has counts of 0 for everything if you want an accurate analysis of the dataframes you passed in. Furthermore, generating the stats object is performed immediately about this function definition in my code, and could be altered for an alternate classification system. 

----
queryStats(stats, weight = "UW", query1="", query2="", query3="", query4="", query5="", query6=""): queryStats() allows you to input a stats 2D list (presumably resulting from runStats), as well as 0 or more queries (a query is a string matching a possible type of result such as "dmn" for default mode network results, etc.) and outputs a list of [increase decrease null] results, in that order. This function is integral to printStats.

The only additional thing you need to do compared to the pre-08/11/2021 code is provide a weighting option as your second parameter:
"UW" - uses unweighted results;
"W-TBI" - uses TBI sample size weighted results;
"W-Total" - uses Total sample size weighted results

----
printStats(stats): printStats() uses queryStats to generate a bunch of pandas dataframes corresponding to relevant outputs for our literature review, making visualization of a ton of different subsections of our results easy and clear.

----
**easyQueryStats(stats, weight = "UW", query1="", query2="", query3="", query4="", query5="", query6="")**: easyQueryStats() works almost exactly like queryStats except it prints out a mini dataframe of your requested statistics, making it easier to use without additional coding. 

**The only additional thing you need to do compared to the pre-08/11/2021 code is provide a weighting option as your second parameter:**
"UW" - uses unweighted results;
"W-TBI" - uses TBI sample size weighted results;
"W-Total" - uses Total sample size weighted results

•This is the function to use if you want to investigate the data in the easiest way possible. Simply go into a new cell and call the function away. An example has been provided in the bottom-most codeblock in this notebook

•Note that if you hop on the notebook and only want to use this function you will still need to run each of the preceding code blocks in order, so that the necessary stats object will be created for you.

•**BE SUPER CAREFUL THAT YOUR QUERIES CORRESPOND TO THESE KEYWORDS OR YOUR COUNTS WON'T BE AS EXPECTED! I did build in a catch for this, but still be careful of spelling.**

nets = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]

results = ["inc", "dec", "null"]

severities = ["mild", "m/mod", "moderate", "mod/sev", "severe", "mix severity", "No Severity"]

ages = ["child", "adolescent", "adult", "mix age", "No Age"]

chronicities = ["Acute", "ac/subac", "subacute", "subac/chron", "chronic", "repsub", "mix cnicity", "No Cnicity"]

types = ["sport", "military", "civilian", "mix Type"]

controls = ["HC", "ISC", "NCC", "TBI+", "Mood", "Other Control"]

**UPDATE 08/11/2021**
Now I am also printing TBI weighted and Total Sample weighted results. These are currently printed right below the previous counts. 
Some additional functions have been provided and some other modifications were made (which no not affect original functionality other than easyQueryStats. 


**-df**


In [41]:
import pandas as pd
import numpy as np
import csv

pd.set_option("display.max_rows", None, "display.max_columns", None)


<i>Setting up directories to CSV files and cleaning the CSVs:</i>

In [42]:
class Directory():
    def __init__(self):
        self.dmn = "PROVIDE DIRECTORY"
        self.ecn = "PROVIDE DIRECTORY"
        self.limbic = "PROVIDE DIRECTORY"
        self.salience = "PROVIDE DIRECTORY"
        self.dan = "PROVIDE DIRECTORY"
        self.van = "PROVIDE DIRECTORY"
        self.sensorimotor = "PROVIDE DIRECTORY"
        self.visual = "PROVIDE DIRECTORY"

dir = Directory()
#edit your directories for the csv files on your system here

baseDir = "/Users/danielfrees/Desktop/LitReview_CSVs"
dir.dmn = baseDir + "/rsfMRI_TBI_Data - DMN.csv"
dir.ecn = baseDir + "/rsfMRI_TBI_Data - ECN.csv"
dir.limbic = baseDir + "/rsfMRI_TBI_Data - Limbic.csv"
dir.salience = baseDir + "/rsfMRI_TBI_Data - Salience.csv"
dir.dan = baseDir + "/rsfMRI_TBI_Data - DAN.csv"
dir.van = baseDir + "/rsfMRI_TBI_Data - VAN.csv"
dir.visual = baseDir + "/rsfMRI_TBI_Data - Visual.csv"
dir.sensorimotor = baseDir + "/rsfMRI_TBI_Data - Sensorimotor.csv"


dmn = pd.read_csv(dir.dmn)
ecn = pd.read_csv(dir.ecn)
limbic = pd.read_csv(dir.limbic)
salience = pd.read_csv(dir.salience)
dan = pd.read_csv(dir.dan)
van = pd.read_csv(dir.van)
visual = pd.read_csv(dir.visual)
sensorimotor = pd.read_csv(dir.sensorimotor)


data = [dmn, ecn, limbic, salience, dan, van, visual, sensorimotor]

#edit the following based on the order of your imported networks. IMPORTANT for the next cell's function to count correctly
dataOrder = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]

for index in range(len(data)):
    data[index] = data[index].fillna('')
    
#BEFORE PROCESSING
#for i in range(len(data)):
#    display(data[i])

#list of words used for decrease, increase, etc. 
decList = ["DECREASE", 'decrease', 'Decrease', 'Dec', 'dec', 'hypo', 'Hypo', 'HYPO', 'Reduce', 'reduce']
incList = ["INCREASE", 'increase', 'Increase', 'Inc', 'inc', 'hyper', 'Hyper', 'HYPER']
nullList = ["NULL", "NO", "No", "no", 'null', 'Null', 'No diff', 'no diff', 'No Diff', 'No sig', 'no sig', 'No Sig', 'NO SIG']
resultWords = decList + incList + nullList


#cleanData function ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----
#cleanData(datalist): cleanData() takes in a list of pandas dataframes (these are generated in my code shortly below the directory inputs), and outputs 
#clean dataframes for the columns relevant to our analysis. I used natural language processing (NLP) to determine what our final verdict for 
#each result/classification was, and then edited the output list of dataframes accordingly.
#Rules: the final determined Result must be written leftmost in its cell so the correct keyword can be identified. 
#Results and Chronicities must be listed for each finding, but other classifications will autofill down wherever they 
#have not been filled in (as we typically only wrote these out for the first finding of a given paper). Certain mispellings 
#and miscapitalizations are allowed, but I did not write a full Trie or other spellchecker, so try to avoid typos as much as possible. 
#I made sure through multiple iterations that this was catching all of our most common mispellings/ miscaps.
def cleanData(datalist):
    for i in range(len(datalist)):
        for index, row in datalist[i].iterrows():
            #CLEAN RESULTS SECTION
            #NOTE: First occurence is counted, because that was how we reported our main findings (always came first). Sometimes the other key words will appear describing a finding which 
            #did not survive due to ANCOVA etc. 
            
            #strip all leading and ending whitespace for the columns to be cleaned (important so that I can check for empty cases vs. error cases)
            row['RESULT'].strip()
            row['TBI Class'].strip()
            row['Severity'].strip()
            row['Age'].strip()
            row['Chronicity'].strip()
            row['Control Type'].strip()            
            
            keyPositions = ((row['RESULT'].find(resWord), resWord) for resWord in resultWords)
            keyPositions = [key for key in keyPositions if key[0] != -1]  #don't want to consider keywords which didn't show up (pos -1)
            try:
                leftmost = min(keyPositions, key = lambda t: t[1]) #anonymous func! yay!
                if leftmost[1] in decList:
                    datalist[i].at[index, 'RESULT'] = "dec"
                elif leftmost[1] in incList:
                    datalist[i].at[index, 'RESULT'] = "inc"
                elif leftmost[1] in nullList:
                    datalist[i].at[index, 'RESULT'] = "null"
            except ValueError as e:
                datalist[i].at[i, 'RESULT'] = "//ERROR//"
            
            
            
            #Clean TBI Class Section
            #allows filling down since we wrote this info only once per paper
            if any(civ in row['TBI Class'] for civ in ["Civilian", "civilian", "civ", "Civ", "CIVILIAN"]):
                datalist[i].at[index, 'TBI Class'] = "civilian"
            elif any(sport in row['TBI Class'] for sport in ["sport", "Sport", "Athlete", "athlete", "ATHLETE", "SPORT"]):
                datalist[i].at[index, 'TBI Class'] = "sport"
            elif any(mil in row['TBI Class'] for mil in ["Military", "military", "blast", "Blast", "MILITARY", "BLAST"]):
                datalist[i].at[index, 'TBI Class'] = "military"
            elif any(mix in row['TBI Class'] for mix in ["mix", "Mix", "MIX"]):
                datalist[i].at[index, 'TBI Class'] = "mix Type"    
            elif index-1 >= 0 and datalist[i].at[index, 'TBI Class'] == "":  #fill down
                datalist[i].at[index, 'TBI Class'] = datalist[i].at[index -1, 'TBI Class']
            else: 
                datalist[i].at[index, 'TBI Class'] = "//ERROR//"
            
                
            #Clean Severity Section
            #allows filling down since we wrote this info only once per paper
            mildTerms = ["MILD", "mild", "Mild"]
            modTerms = ["MOD", "mod", "Mod"]
            sevTerms = ["SEV", "sev", "Sev"]
            mixTerms = ["MIX", "mix", "Mix", "all", "All", "ALL"]
            hasMild = False
            hasMod = False
            hasSev = False
            hasMix = False
            if any(mild in row['Severity'] for mild in mildTerms):
                hasMild = True
            if any(mod in row['Severity'] for mod in modTerms):  #we sometimes used shortenings for moderate/severe
                hasMod = True
            if any(sev in row['Severity'] for sev in sevTerms):
                hasSev = True
            if any(mix in row['Severity'] for mix in mixTerms):
                hasMix = True
                
            if row['Severity'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'Severity'] = datalist[i].at[index -1, 'Severity'] #fill down
            elif not row['Severity'] == "" and not any(keyw in row['Severity'] for keyw in mildTerms + modTerms + sevTerms + mixTerms):  #check for errors
                datalist[i].at[index, 'Severity'] = "//ERROR//"
            elif hasMild and hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "m/mod"
            elif not hasMild and hasMod and hasSev:
                datalist[i].at[index, 'Severity'] = "mod/sev"
            elif hasMild and not hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "mild"
            elif not hasMild and hasMod and not hasSev:
                datalist[i].at[index, 'Severity'] = "moderate"
            elif not hasMild and not hasMod and hasSev:
                datalist[i].at[index, 'Severity'] = "severe"
            elif hasMix:
                datalist[i].at[index, 'Severity'] = "mix severity"
            elif not row['Severity'] == "":
                datalist[i].at[index, 'Severity'] = "mix severity"
            
            
            #clean Age section
            #allows filling down since we wrote this info only once per paper
            adultList = ["ADULT", "adult", 'Adult']
            adolescentList = ["ADOL", "adolescent", "Adolescent"]
            childList = ["CHILD, ""child", "Child"]
            mixList = ["MIX", "mix", "Mix"]
            if any(mix in row['Age'] for mix in mixList) or (any(adult in row['Age'] for adult in adultList) and (any(adolescent in row['Age'] for adolescent in adolescentList) or any(child in row['Age'] for child in childList))) or (any(adolescent in row['Age'] for adolescent in adolescentList) and any(child in row['Age'] for child in childList)):
                datalist[i].at[index, 'Age'] = "mix age"
            elif any(adult in row['Age'] for adult in adultList):
                datalist[i].at[index, 'Age'] = 'adult'
            elif any(adolescent in row['Age'] for adolescent in adolescentList):
                datalist[i].at[index, 'Age'] = "adolescent"
            elif any(child in row['Age'] for child in childList):
                datalist[i].at[index, 'Age'] = "child"
            elif row['Age'] == "" and index-1 >= 0:  #fill down
                datalist[i].at[index, 'Age'] = datalist[i].at[index -1, 'Age'] #fill down
            else:
                datalist[i].at[index, 'Age'] = "//ERROR//"

            #clean Chronicity Section 
            #note: no filling, each finding must have its own chronicity
            acuteList = ["ACUTE", "acute", "Acute"]
            subacuteList = ["SUBACUTE", "subacute", "Subacute"]
            chronicList = ["CHRONIC", "chronic", "Chronic"]
            mixedList = ["MIX", "mix", "Mix"]
            repSubList = ["REP", "SUBCON", "repetitive", "Repetitive", "subconcussive", "Subconcussive"]
            hasAcute = False
            hasSubacute = False
            hasChronic = False
            hasMix = False
            hasRepSub = False
            
            #dealing with the fact that acute exists within subacute
            numAcutes = 0
            for acute in acuteList:
                numAcutes += row['Chronicity'].count(acute) 
            
            #if acute and not subacute we have an actual acute, if acute shows up multiple places we should have acute
            #in the form acute/subacute
            if numAcutes == 1 and not any (sub in row['Chronicity'] for sub in subacuteList) or numAcutes > 1:
                hasAcute = True
                
            if any (sub in row['Chronicity'] for sub in subacuteList):
                hasSubacute = True
            if any (chron in row['Chronicity'] for chron in chronicList):
                hasChronic = True
            if any (mix in row['Chronicity'] for mix in mixedList):
                hasMix = True
            if any (repSub in row['Chronicity'] for repSub in repSubList):
                hasRepSub = True
            if hasRepSub:
                datalist[i].at[index, 'Chronicity'] = "repsub"
            elif hasMix or (hasAcute and hasChronic):
                datalist[i].at[index, 'Chronicity'] = "mix cnicity"
            elif hasAcute and hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "ac/subac"
            elif not hasAcute and hasSubacute and hasChronic:
                datalist[i].at[index, 'Chronicity'] = "subac/chron"
            elif not hasAcute and not hasSubacute and hasChronic:
                datalist[i].at[index, 'Chronicity'] = "chronic"
            elif hasAcute and not hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "Acute"
            elif not hasAcute and hasSubacute and not hasChronic:
                datalist[i].at[index, 'Chronicity'] = "subacute"   
            else:
                datalist[i].at[index, 'Chronicity'] = "//ERROR//"
                
            #clean Control Section
            #allows filling down since we wrote this info only once per paper
            healthyList = ["HC", "healthy", "Healthy", "HEALTHY"]
            inSportList = ["ISC", "In-sport", "in-sport", "IN-SPORT", "IN SPORT"]
            nonContactList = ["NCC", "Non-contact", "non-contact", "NON-CONTACT", "non-contact", "Non-Contact", "NON CONTACT"]
            tbiPlusMoodList = ["TBI+", "tbi+", "Tbi+", "TBI plus", "tbi plus", "Tbi plus", "tbi +", "Tbi +", "TBI PLUS"]
            moodList = ["mood", "Mood", "anxiety", "Anxiety", "MOOD", "ANXIETY"]
            otherList =  ["other", "Other", "Orthopedic", "orthopedic", "military", "Military", "OTHER", "ORTHO", "MILIT"]
            if row['Control Type'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'Control Type'] = datalist[i].at[index -1, 'Control Type']
            elif any(healthy in row['Control Type'] for healthy in healthyList):
                datalist[i].at[index, 'Control Type'] = "HC"
            elif any(inSport in row['Control Type'] for inSport in inSportList):
                datalist[i].at[index, 'Control Type'] = "ISC"
            elif any(nonCon in row['Control Type'] for nonCon in nonContactList):
                datalist[i].at[index, 'Control Type'] = "NCC"    
            elif any(tbiPlus in row['Control Type'] for tbiPlus in tbiPlusMoodList):
                datalist[i].at[index, 'Control Type'] = "TBI+"
            elif any(mood in row['Control Type'] for mood in moodList):
                datalist[i].at[index, 'Control Type'] = "Mood"
            elif any(other in row['Control Type'] for other in otherList):
                datalist[i].at[index, 'Control Type'] = "Other Control"    
            else: 
                datalist[i].at[index, 'Control Type'] = "//ERROR//"
                
            #clean up sample sizes (fill down), and put errors for missing info
            if row['TBI (n)'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'TBI (n)'] = datalist[i].at[index-1, 'TBI (n)']
            elif row['TBI (n)'] == "X": 
                datalist[i].at[index, 'TBI (n)'] = "//ERROR//"
            
            if row['HC (n)'] == "" and index-1 >= 0:   #fill down
                datalist[i].at[index, 'HC (n)'] = datalist[i].at[index-1, 'HC (n)']
            elif row['HC (n)'] == "X": 
                datalist[i].at[index, 'HC (n)'] = "//ERROR//"
                
    return datalist      
##end cleanData function ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ----


#actually clean the data
data = cleanData(data)

#AFTER PROCESSING: (unhash code if you want to print the outputs) 


#netLabels = ["DMN", "ECN", "Limbic", "Salience", "DAN", "VAN", "Visual", "Sensorimotor"]
#for i in range(len(data)):
    #print(netLabels[i] + ":")
    #display(data[i])
            
            

In [43]:
#COUNT EVERYTHING UP AND REPORT IT!
#Note: As of latest update, stats now holds UW count, TBI sample size weighted counted, total sample size weighted count (in that order)

classifications = []   #to hold all possible specific classifications

#made it so that no string was contained within another string to make for simpler searching when coming up with statistics
nets = ["dmn", "ecn", "limb", "sn", "dan", "van", "vis", "smn"]
results = ["inc", "dec", "null"]
severities = ["mild", "m/mod", "moderate", "mod/sev", "severe", "mix severity", "No Severity"]
ages = ["child", "adolescent", "adult", "mix age", "No Age"]
chronicities = ["Acute", "ac/subac", "subacute", "subac/chron", "chronic", "repsub", "mix cnicity", "No Cnicity"]
types = ["sport", "military", "civilian", "mix Type"]
controls = ["HC", "ISC", "NCC", "TBI+", "Mood", "Other Control"]

for net in nets:
    for result in results:
        for severity in severities:
            for age in ages:
                for chronicity in chronicities:
                    for typ in types:
                        for control in controls:
                            classifications.append(net + "_" + result + "_" + severity + "_" + age +
                                                   "_" + chronicity + "_" + typ + "_" + control)
                            
stats = [[classi, 0, 0, 0] for classi in classifications]                               

#print(len(classifications)) 


#runStats() function----------------------------------------------------------------------------------------------------------------------------
#runStats(datalist, stats): runStats() takes in a list of pandas dataframes (these should be the cleaned ones output from cleanData()) as well 
#as a 2D 'stats' list (a list of size 2 lists, where each size 2 list is a pair: a classification, and a count of how 
#many times that classification shows up in the data (this should start at 0 for all classifications the same way I initially 
#set up my stats object). It outputs a modified 2D 'stats' list with the counts added up for each classification after parsing through the provided datalist.
#Rules: Mostly just be careful to pass a stats 2D list which has counts of 0 for everything if you want an accurate analysis of 
#the dataframes you passed in. Furthermore, generating the stats object is performed immediately about this function definition in my code, 
#and could be altered for an alternate classification system.
#Also note: errors should already be checked! they can be allowed to 
#remain if desired for missing info on severity, age, chronicity. Other rows must be figured out. Data is required.
def runStats(datalist, stats):
    for i in range(len(datalist)):
        for index, row in datalist[i].iterrows():
            classif = ""
            classif += dataOrder[i]
            classif += "_" + row['RESULT']
            
            
            if row['Severity'] == "//ERROR//":
                classif += "_" + "No Severity"
            else:
                classif += "_" + row['Severity']
                
            if row['Age'] == "//ERROR//":
                classif += "_" + "No Age"
            else:
                classif += "_" + row['Age']
                
            if row['Chronicity'] == "//ERROR//":
                classif += "_" + "No Cnicity"
            else:
                classif += "_" + row['Chronicity']
                
            classif += "_" + row['TBI Class']
            classif += "_" + row['Control Type']
            
            for stat in stats:
                if stat[0] == classif:
                    stat[1] += 1
                    if row['TBI (n)'] != "//ERROR//":
                        stat[2] += int(row['TBI (n)'])
                    if row['TBI (n)'] != "//ERROR//" and row['HC (n)'] != "//ERROR//":
                        stat[3] += ( int(row['TBI (n)']) + int(row['HC (n)']) )           

runStats(data, stats)
#end runStats() function----------------------------------------------------------------------------------------------------------------------------


#queryStats() function----------------------------------------------------------------------------------------------------------------------------
#queryStats(stats, weight = "UW", query1="", query2="", query3="", query4="", query5="", query6=""): queryStats() allows you to input a stats 2D list 
#(presumably resulting from runStats), as well as 0 to 6 queries (a query is a string matching a possible type of result such as "dmn" for 
#default mode network results, etc.) and outputs a list of [increase decrease null] results, in that order. This function is integral to printStats.
def queryStats(stats, weight = "UW", query1="", query2="", query3="", query4="", query5="", query6=""):
    count_inc = 0
    count_dec = 0
    count_null = 0
    for stat in stats:
        if "inc" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                if weight == "UW":
                    count_inc += stat[1]
                elif weight == "W-TBI":
                    count_inc += stat[2]
                elif weight == "W-Total":
                    count_inc += stat[3]
        if "dec" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                if weight == "UW":
                    count_dec += stat[1]
                elif weight == "W-TBI":
                    count_dec += stat[2]
                elif weight == "W-Total":
                    count_dec += stat[3]
        if "null" in stat[0] and query1 in stat[0] and query2 in stat[0] and query3 in stat[0] \
            and query4 in stat[0] and query5 in stat[0] and query6 in stat[0]:
                if weight == "UW":
                    count_null += stat[1]
                elif weight == "W-TBI":
                    count_null += stat[2]
                elif weight == "W-Total":
                    count_null += stat[3]
    return [count_inc, count_dec, count_null]
#end queryStats() function----------------------------------------------------------------------------------------------------------------------------


#printStats() function----------------------------------------------------------------------------------------------------------------------------
#printStats(stats): printStats() uses queryStats to generate a bunch of pandas dataframes corresponding to relevant outputs for our literature review,
#making visualization of a ton of different subsections of our results easy and clear.
#A weight must be provided as described for queryStats(), otherwise the default is unweighted results.
def printStats(stats, weight = "UW"):
    print("------------------------------\n")
    print('Starting ' + weight + ' Counts...')
    print('\n')
    
    #network totals:
    
    #dmn
    dmn = queryStats(stats, weight, "dmn")

    #ecn        
    ecn = queryStats(stats, weight, "ecn")
    
    #Limbic
    limb = queryStats(stats, weight, "limb")
    
    #Salience/Ventral
    sn = queryStats(stats, weight, "sn")
    
    van = queryStats(stats, weight, "van")
            
    snvan = [sn[0] + van[0], sn[1] + van[1], sn[2] + van[2]]
    
    
    #DAN
    dan = queryStats(stats, weight, "dan")
    
    #Visual
    vis = queryStats(stats, weight, "vis")
    
    #Sensorimotor
    smn = queryStats(stats, weight, "smn")
    
    total_inc = dmn[0] + ecn[0] + limb[0] + snvan[0] + dan[0] + vis[0] + smn[0]
    total_dec = dmn[1] + ecn[1] + limb[1] + snvan[1] + dan[1] + vis[1] + smn[1]
    total_null = dmn[2] + ecn[2] + limb[2] + snvan[2] + dan[2] + vis[2] + smn[2]
    
    data_totals = {'Increase':[dmn[0], ecn[0], limb[0], snvan[0], dan[0], vis[0], smn[0], total_inc], 
                'Decrease':[dmn[1], ecn[1], limb[1], snvan[1], dan[1], vis[1], smn[1], total_dec], 
                'Null':[dmn[2], ecn[2], limb[2], snvan[2], dan[2], vis[2], smn[2], total_null]}        
    df_totals = pd.DataFrame(data_totals, index = ['DMN', 'ECN', 'Limbic', 'Salience/VAN', 'DAN', 'Visual', 'Sensorimotor', 'Total'])
    
    print("Network Totals (" + weight +  "): ")
    display(df_totals)
    
    #now in detail..
    detailed_index = ['All', 'Mild', 'Mild/Moderate', 'Moderate', 'Moderate/Severe', 
             'Severe', 'Mixed Severity', 'No Severity Stated', 'Child', 'Adolescent', 'Adult', 'Mixed Age', 'No Age Stated', 
             'Acute', 'Acute/Subacute', 'Subacute', 'Subacute/Chronic', 'Chronic', 'Repetitive Subconcussive',
             'Mixed Chronicity', 'No Chronicity Stated', 'Sport', 'Military', 'Civilian', 'Mixed Type', 
             'Healthy Control', 'No Contact Control', 'In Sport Control', 'Other Control']
    
    
    #All networks in Detail-------------------:
    print("All Networks in Detail (" + weight +  "):")

    all = [total_inc, total_dec, total_null]
    all_mild = queryStats(stats, weight, "mild")
    all_mildMod = queryStats(stats, weight, "m/mod")
    all_mod = queryStats(stats, weight, "moderate")
    all_modSev = queryStats(stats, weight, "mod/sev")
    all_sev = queryStats(stats, weight, "severe")
    all_mixSev = queryStats(stats, weight, "mix severity")
    all_noSev = queryStats(stats, weight, "No Severity")
    all_child = queryStats(stats, weight, "child")
    all_adolescent = queryStats(stats, weight, "adolescent")
    all_adult = queryStats(stats, weight, "adult")
    all_mixAge = queryStats(stats, weight, "mix age")
    all_noAge = queryStats(stats, weight, "No Age")
    all_acute = queryStats(stats, weight, "Acute")
    all_acuteSub = queryStats(stats, weight, "ac/subac")
    all_subacute = queryStats(stats, weight, "subacute")
    all_subChron = queryStats(stats, weight, "subac/chron")
    all_chronic = queryStats(stats, weight, "chronic")
    all_repSub = queryStats(stats, weight, "repsub")
    all_mixChron = queryStats(stats, weight, "mix cnicity")
    all_noChron = queryStats(stats, weight, "No Cnicity")
    all_sport = queryStats(stats, weight, "sport")
    all_military = queryStats(stats, weight, "military")
    all_civilian = queryStats(stats, weight, "civilian")
    all_mixType = queryStats(stats, weight, "mix Type")
    all_healthyControl = queryStats(stats, weight, "HC")
    all_noContactControl = queryStats(stats, weight, "NCC")
    all_inSportControl = queryStats(stats, weight, "ISC")
    all_otherControl = queryStats(stats, weight, "Other Control")
    
    all_categories = [all, all_mild, all_mildMod, all_mod, all_modSev, all_sev, 
                  all_mixSev, all_noSev, all_child, all_adolescent, all_adult,
                 all_mixAge, all_noAge, all_acute, all_acuteSub, all_subacute, all_subChron, all_chronic, all_repSub, 
                 all_mixChron, all_noChron, all_sport, all_military, all_civilian, all_mixType, 
                 all_healthyControl, all_noContactControl, all_inSportControl, all_otherControl]
    
    all_incList = []
    all_decList = []
    all_nullList = []
    for cat in all_categories:
        all_incList.append(cat[0])
        all_decList.append(cat[1])
        all_nullList.append(cat[2])
    
    data_all = {'Increase': all_incList, 
                'Decrease': all_decList, 
                'Null': all_nullList}        
    df_all = pd.DataFrame(data_all, index = detailed_index)
    display(df_all)
    
    #BY SEVERITY AND CHRONICITY (ALL)
    print("Results by Severity & Chronicity (" + weight +  ")")
    mildAcute = queryStats(stats, weight, "mild", "Acute")
    mildAcuteSubacute = queryStats(stats, weight, "mild", "ac/subac")
    mildSubacute = queryStats(stats, weight, "mild", "subacute")
    mildSubacuteChronic = queryStats(stats, weight, "mild", "subac/chron")
    mildChronic = queryStats(stats, weight, "mild", "chronic")
    mildMixChron = queryStats(stats, weight, "mild", "mix cnicity")
    mildNoChron = queryStats(stats, weight, "mild", "No Cnicity")
    
    mildModAcute = queryStats(stats, weight, "m/mod", "Acute") 
    mildModAcuteSubacute = queryStats(stats, weight, "m/mod", "ac/subac")
    mildModSubacute = queryStats(stats, weight, "m/mod", "subacute")
    mildModSubacuteChronic = queryStats(stats, weight, "m/mod", "subac/chron")
    mildModChronic = queryStats(stats, weight, "m/mod", "chronic")
    mildModMixChron = queryStats(stats, weight, "m/mod", "mix cnicity")
    mildModNoChron = queryStats(stats, weight, "m/mod", "No Cnicity")
    
    modAcute = queryStats(stats, weight, "moderate", "Acute") 
    modAcuteSubacute = queryStats(stats, weight, "moderate", "ac/subac")
    modSubacute = queryStats(stats, weight, "moderate", "subacute")
    modSubacuteChronic = queryStats(stats, weight, "moderate", "subac/chron")
    modChronic = queryStats(stats, weight, "moderate", "chronic")
    modMixChron = queryStats(stats, weight, "moderate", "mix cnicity")
    modNoChron = queryStats(stats, weight, "moderate", "No Cnicity")
    
    modSevereAcute = queryStats(stats, weight, "mod/sev", "Acute") 
    modSevereAcuteSubacute = queryStats(stats, weight, "mod/sev", "ac/subac")
    modSevereSubacute = queryStats(stats, weight, "mod/sev", "subacute")
    modSevereSubacuteChronic = queryStats(stats, weight, "mod/sev", "subac/chron")
    modSevereChronic = queryStats(stats, weight, "mod/sev", "chronic")
    modSevereMixChron = queryStats(stats, weight, "mod/sev", "mix cnicity")
    modSevereNoChron = queryStats(stats, weight, "mod/sev", "No Cnicity")
    
    severeAcute = queryStats(stats, weight, "severe", "Acute") 
    severeAcuteSubacute = queryStats(stats, weight, "severe", "ac/subac")
    severeSubacute = queryStats(stats, weight, "severe", "subacute")
    severeSubacuteChronic = queryStats(stats, weight, "severe", "subac/chron")
    severeChronic = queryStats(stats, weight, "severe", "chronic")
    severeMixChron = queryStats(stats, weight, "severe", "mix cnicity")
    severeNoChron = queryStats(stats, weight, "severe", "No Cnicity")
    
    mixedSevAcute = queryStats(stats, weight, "mix severity", "Acute") 
    mixedSevAcuteSubacute = queryStats(stats, weight, "mix severity", "ac/subac")
    mixedSevSubacute = queryStats(stats, weight, "mix severity", "subacute")
    mixedSevSubacuteChronic = queryStats(stats, weight, "mix severity", "subac/chron")
    mixedSevChronic = queryStats(stats, weight, "mix severity", "chronic")
    mixedSevMixChron = queryStats(stats, weight, "mix severity", "mix cnicity")
    mixedSevNoChron = queryStats(stats, weight, "mix severity", "No Cnicity")
    
    noSevAcute = queryStats(stats, weight, "No Severity", "Acute") 
    noSevAcuteSubacute = queryStats(stats, weight, "No Severity", "ac/subac")
    noSevSubacute = queryStats(stats, weight, "No Severity", "subacute")
    noSevSubacuteChronic = queryStats(stats, weight, "No Severity", "subac/chron")
    noSevChronic = queryStats(stats, weight, "No Severity", "chronic")
    noSevMixChron = queryStats(stats, weight, "No Severity", "mix cnicity")
    noSevNoChron = queryStats(stats, weight, "No Severity", "No Cnicity")
    
    sevChron_list = [mildAcute, mildAcuteSubacute, mildSubacute, mildSubacuteChronic, mildChronic, mildMixChron,
                    mildNoChron, mildModAcute, mildModAcuteSubacute, mildModSubacute, mildModSubacuteChronic, 
                    mildModChronic, mildModMixChron, mildModNoChron, modAcute, modAcuteSubacute, modSubacute,
                    modSubacuteChronic, modChronic, modMixChron, modNoChron, modSevereAcute, modSevereAcuteSubacute,
                    modSevereSubacute, modSevereSubacuteChronic, modSevereChronic, modSevereMixChron, modSevereNoChron,
                    severeAcute, severeAcuteSubacute, severeSubacute, severeSubacuteChronic, severeChronic, severeMixChron,
                    severeNoChron, mixedSevAcute, mixedSevAcuteSubacute, mixedSevSubacute, mixedSevSubacuteChronic,
                    mixedSevChronic, mixedSevMixChron, mixedSevNoChron, noSevAcute, noSevAcuteSubacute, noSevSubacute, noSevSubacuteChronic,
                    noSevChronic, noSevMixChron, noSevNoChron]
    
    sevChron_incList = []
    sevChron_decList = []
    sevChron_nullList = []
    
    for sevChron in sevChron_list:
        sevChron_incList.append(sevChron[0])
        sevChron_decList.append(sevChron[1])
        sevChron_nullList.append(sevChron[2])
    
    sevChron_index = ["Mild Acute", "Mild Acute/Subacute", "Mild Subacute", "Mild Subacute/Chronic", "Mild Chronic", "Mild Mixed Chronicity", "Mild + No Chronicity Stated", 
                      "Mild/Mod Acute", "Mild/Mod Acute/Subacute", "Mild/Mod Subacute", "Mild/Mod Subacute/Chronic", "Mild/Mod Chronic", "Mild/Mod Mixed Chronicity", "Mild/Mod + No Chronicity Stated", 
                      "Moderate Acute", "Moderate Acute/Subacute", "Moderate Subacute", "Moderate Subacute/Chronic", "Moderate Chronic", "Moderate Mixed Chronicity", "Moderate + No Chronicity Stated",
                      "Mod/Sev Acute", "Mod/Sev Acute/Subacute", "Mod/Sev Subacute", "Mod/Sev Subacute/Chronic", "Mod/Sev Chronic", "Mod/Sev Mixed Chronicity", "Mod/Sev + No Chronicity Stated", 
                      "Severe Acute", "Severe Acute/Subacute", "Severe Subacute", "Severe Subacute/Chronic", "Severe Chronic", "Severe Mixed Chronicity", "Severe + No Chronicity Stated",
                      "Mixed Sev. Acute", "Mixed Sev. Acute/Subacute", "Mixed Sev. Subacute", "Mixed Sev. Subacute/Chronic", "Mixed Sev. Chronic", "Mixed Sev. Mixed Chronicity", "Mixed Sev. + No Chronicity Stated", 
                      "No Sev. Stated Acute", "No Sev. Stated Acute/Subacute", "No Sev. Stated Subacute", "No Sev. Stated Subacute/Chronic", "No Sev. Stated Chronic", "No Sev. Stated Mixed Chronicity", "No Sev. Stated + No Chronicity Stated"] 
    
    data_sevChron = {'Increase': sevChron_incList, 
                'Decrease': sevChron_decList, 
                'Null': sevChron_nullList}        
    
    df_sevChron = pd.DataFrame(data_sevChron, sevChron_index)
    display(df_sevChron)
    
    #DMN------------------------------:
    print("DMN in Detail (" + weight +  "):")
    
    dmn_mild = queryStats(stats, weight, "dmn", "mild")
    dmn_mildMod = queryStats(stats, weight, "dmn", "m/mod")
    dmn_mod = queryStats(stats, weight, "dmn", "moderate")
    dmn_modSev = queryStats(stats, weight, "dmn", "mod/sev")
    dmn_sev = queryStats(stats, weight, "dmn", "severe")
    dmn_mixSev = queryStats(stats, weight, "dmn", "mix severity")
    dmn_noSev = queryStats(stats, weight, "dmn", "No Severity")
    dmn_child = queryStats(stats, weight, "dmn", "child")
    dmn_adolescent = queryStats(stats, weight, "dmn", "adolescent")
    dmn_adult = queryStats(stats, weight, "dmn", "adult")
    dmn_mixAge = queryStats(stats, weight, "dmn", "mix age")
    dmn_noAge = queryStats(stats, weight, "dmn", "No Age")
    dmn_acute = queryStats(stats, weight, "dmn", "Acute")
    dmn_acuteSub = queryStats(stats, weight, "dmn", "ac/subac")
    dmn_subacute = queryStats(stats, weight, "dmn", "subacute")
    dmn_subChron = queryStats(stats, weight, "dmn", "subac/chron")
    dmn_chronic = queryStats(stats, weight, "dmn", "chronic")
    dmn_repSub = queryStats(stats, weight, "dmn", "repsub")
    dmn_mixChron = queryStats(stats, weight, "dmn", "mix cnicity")
    dmn_noChron = queryStats(stats, weight, "dmn", "No Cnicity")
    dmn_sport = queryStats(stats, weight, "dmn", "sport")
    dmn_military = queryStats(stats, weight, "dmn", "military")
    dmn_civilian = queryStats(stats, weight, "dmn", "civilian")
    dmn_mixType = queryStats(stats, weight, "dmn", "mix Type")
    dmn_healthyControl = queryStats(stats, weight, "dmn", "HC")
    dmn_noContactControl = queryStats(stats, weight, "dmn", "NCC")
    dmn_inSportControl = queryStats(stats, weight, "dmn", "ISC")
    dmn_otherControl = queryStats(stats, weight, "dmn", "Other Control")
    
    dmn_categories = [dmn, dmn_mild, dmn_mildMod, dmn_mod, dmn_modSev, dmn_sev, 
                  dmn_mixSev, dmn_noSev, dmn_child, dmn_adolescent, dmn_adult,
                 dmn_mixAge, dmn_noAge, dmn_acute, dmn_acuteSub, dmn_subacute, dmn_subChron, dmn_chronic, dmn_repSub, 
                 dmn_mixChron, dmn_noChron, dmn_sport, dmn_military, dmn_civilian, dmn_mixType, 
                 dmn_healthyControl, dmn_noContactControl, dmn_inSportControl, dmn_otherControl]
    
    dmn_incList = []
    for cat in dmn_categories:
        dmn_incList.append(cat[0])
    dmn_decList = []
    for cat in dmn_categories:
        dmn_decList.append(cat[1])
    dmn_nullList = []
    for cat in dmn_categories:
        dmn_nullList.append(cat[2])
    
    data_dmn = {'Increase': dmn_incList, 
                'Decrease': dmn_decList, 
                'Null': dmn_nullList}        
    df_dmn = pd.DataFrame(data_dmn, index = detailed_index)
    display(df_dmn)
    
    #ECN------------------------------:
    print("ECN in Detail (" + weight +  "):")
    
    ecn_mild = queryStats(stats, weight, "ecn", "mild")
    ecn_mildMod = queryStats(stats, weight, "ecn", "m/mod")
    ecn_mod = queryStats(stats, weight, "ecn", "moderate")
    ecn_modSev = queryStats(stats, weight, "ecn", "mod/sev")
    ecn_sev = queryStats(stats, weight, "ecn", "severe")
    ecn_mixSev = queryStats(stats, weight, "ecn", "mix severity")
    ecn_noSev = queryStats(stats, weight, "ecn", "No Severity")
    ecn_child = queryStats(stats, weight, "ecn", "child")
    ecn_adolescent = queryStats(stats, weight, "ecn", "adolescent")
    ecn_adult = queryStats(stats, weight, "ecn", "adult")
    ecn_mixAge = queryStats(stats, weight, "ecn", "mix age")
    ecn_noAge = queryStats(stats, weight, "ecn", "No Age")
    ecn_acute = queryStats(stats, weight, "ecn", "Acute")
    ecn_acuteSub = queryStats(stats, weight, "ecn", "ac/subac")
    ecn_subacute = queryStats(stats, weight, "ecn", "subacute")
    ecn_subChron = queryStats(stats, weight, "ecn", "subac/chron")
    ecn_chronic = queryStats(stats, weight, "ecn", "chronic")
    ecn_repSub = queryStats(stats, weight, "ecn", "repsub")
    ecn_mixChron = queryStats(stats, weight, "ecn", "mix cnicity")
    ecn_noChron = queryStats(stats, weight, "ecn", "No Cnicity")
    ecn_sport = queryStats(stats, weight, "ecn", "sport")
    ecn_military = queryStats(stats, weight, "ecn", "military")
    ecn_civilian = queryStats(stats, weight, "ecn", "civilian")
    ecn_mixType = queryStats(stats, weight, "ecn", "mix Type")
    ecn_healthyControl = queryStats(stats, weight, "ecn", "HC")
    ecn_noContactControl = queryStats(stats, weight, "ecn", "NCC")
    ecn_inSportControl = queryStats(stats, weight, "ecn", "ISC")
    ecn_otherControl = queryStats(stats, weight, "ecn", "Other Control")
    
    ecn_categories = [ecn, ecn_mild, ecn_mildMod, ecn_mod, ecn_modSev, ecn_sev, 
                  ecn_mixSev, ecn_noSev, ecn_child, ecn_adolescent, ecn_adult,
                 ecn_mixAge, ecn_noAge, ecn_acute, ecn_acuteSub, ecn_subacute, ecn_subChron, ecn_chronic, ecn_repSub, 
                 ecn_mixChron, ecn_noChron, ecn_sport, ecn_military, ecn_civilian, ecn_mixType, 
                 ecn_healthyControl, ecn_noContactControl, ecn_inSportControl, ecn_otherControl]
    
    ecn_incList = []
    for cat in ecn_categories:
        ecn_incList.append(cat[0])
    ecn_decList = []
    for cat in ecn_categories:
        ecn_decList.append(cat[1])
    ecn_nullList = []
    for cat in ecn_categories:
        ecn_nullList.append(cat[2])
    
    data_ecn = {'Increase': ecn_incList, 
                'Decrease': ecn_decList, 
                'Null': ecn_nullList}        
    df_ecn = pd.DataFrame(data_ecn, index = detailed_index)
    display(df_ecn)
    
    #Limbic------------------------------:
    print("Limbic in Detail (" + weight +  "):")
    
    limb_mild = queryStats(stats, weight, "limb", "mild")
    limb_mildMod = queryStats(stats, weight, "limb", "m/mod")
    limb_mod = queryStats(stats, weight, "limb", "moderate")
    limb_modSev = queryStats(stats, weight, "limb", "mod/sev")
    limb_sev = queryStats(stats, weight, "limb", "severe")
    limb_mixSev = queryStats(stats, weight, "limb", "mix severity")
    limb_noSev = queryStats(stats, weight, "limb", "No Severity")
    limb_child = queryStats(stats, weight, "limb", "child")
    limb_adolescent = queryStats(stats, weight, "limb", "adolescent")
    limb_adult = queryStats(stats, weight, "limb", "adult")
    limb_mixAge = queryStats(stats, weight, "limb", "mix age")
    limb_noAge = queryStats(stats, weight, "limb", "No Age")
    limb_acute = queryStats(stats, weight, "limb", "Acute")
    limb_acuteSub = queryStats(stats, weight, "limb", "ac/subac")
    limb_subacute = queryStats(stats, weight, "limb", "subacute")
    limb_subChron = queryStats(stats, weight, "limb", "subac/chron")
    limb_chronic = queryStats(stats, weight, "limb", "chronic")
    limb_repSub = queryStats(stats, weight, "limb", "repsub")
    limb_mixChron = queryStats(stats, weight, "limb", "mix cnicity")
    limb_noChron = queryStats(stats, weight, "limb", "No Cnicity")
    limb_sport = queryStats(stats, weight, "limb", "sport")
    limb_military = queryStats(stats, weight, "limb", "military")
    limb_civilian = queryStats(stats, weight, "limb", "civilian")
    limb_mixType = queryStats(stats, weight, "limb", "mix Type")
    limb_healthyControl = queryStats(stats, weight, "limb", "HC")
    limb_noContactControl = queryStats(stats, weight, "limb", "NCC")
    limb_inSportControl = queryStats(stats, weight, "limb", "ISC")
    limb_otherControl = queryStats(stats, weight, "limb", "Other Control")
    
    limb_categories = [limb, limb_mild, limb_mildMod, limb_mod, limb_modSev, limb_sev, 
                  limb_mixSev, limb_noSev, limb_child, limb_adolescent, limb_adult,
                 limb_mixAge, limb_noAge, limb_acute, limb_acuteSub, limb_subacute, limb_subChron, limb_chronic, limb_repSub, 
                 limb_mixChron, limb_noChron, limb_sport, limb_military, limb_civilian, limb_mixType, 
                 limb_healthyControl, limb_noContactControl, limb_inSportControl, limb_otherControl]
    
    limb_incList = []
    for cat in limb_categories:
        limb_incList.append(cat[0])
    limb_decList = []
    for cat in limb_categories:
        limb_decList.append(cat[1])
    limb_nullList = []
    for cat in limb_categories:
        limb_nullList.append(cat[2])
    
    data_limb = {'Increase': limb_incList, 
                'Decrease': limb_decList, 
                'Null': limb_nullList}        
    df_limb = pd.DataFrame(data_limb, index = detailed_index)
    display(df_limb)
    
    #Salience/VAN------------------------:
    print("Salience/VAN in Detail (" + weight +  "):")
    
    #Salience
    sn_mild = queryStats(stats, weight, "sn", "mild")
    sn_mildMod = queryStats(stats, weight, "sn", "m/mod")
    sn_mod = queryStats(stats, weight, "sn", "moderate")
    sn_modSev = queryStats(stats, weight, "sn", "mod/sev")
    sn_sev = queryStats(stats, weight, "sn", "severe")
    sn_mixSev = queryStats(stats, weight, "sn", "mix severity")
    sn_noSev = queryStats(stats, weight, "sn", "No Severity")
    sn_child = queryStats(stats, weight, "sn", "child")
    sn_adolescent = queryStats(stats, weight, "sn", "adolescent")
    sn_adult = queryStats(stats, weight, "sn", "adult")
    sn_mixAge = queryStats(stats, weight, "sn", "mix age")
    sn_noAge = queryStats(stats, weight, "sn", "No Age")
    sn_acute = queryStats(stats, weight, "sn", "Acute")
    sn_acuteSub = queryStats(stats, weight, "sn", "ac/subac")
    sn_subacute = queryStats(stats, weight, "sn", "subacute")
    sn_subChron = queryStats(stats, weight, "sn", "subac/chron")
    sn_chronic = queryStats(stats, weight, "sn", "chronic")
    sn_repSub = queryStats(stats, weight, "sn", "repsub")
    sn_mixChron = queryStats(stats, weight, "sn", "mix cnicity")
    sn_noChron = queryStats(stats, weight, "sn", "No Cnicity")
    sn_sport = queryStats(stats, weight, "sn", "sport")
    sn_military = queryStats(stats, weight, "sn", "military")
    sn_civilian = queryStats(stats, weight, "sn", "civilian")
    sn_mixType = queryStats(stats, weight, "sn", "mix Type")
    sn_healthyControl = queryStats(stats, weight, "sn", "HC")
    sn_noContactControl = queryStats(stats, weight, "sn", "NCC")
    sn_inSportControl = queryStats(stats, weight, "sn", "ISC")
    sn_otherControl = queryStats(stats, weight, "sn", "Other Control")
    
    sn_categories = [sn, sn_mild, sn_mildMod, sn_mod, sn_modSev, sn_sev, 
                  sn_mixSev, sn_noSev, sn_child, sn_adolescent, sn_adult,
                 sn_mixAge, sn_noAge, sn_acute, sn_acuteSub, sn_subacute, sn_subChron, sn_chronic, sn_repSub, 
                 sn_mixChron, sn_noChron, sn_sport, sn_military, sn_civilian, sn_mixType, 
                 sn_healthyControl, sn_noContactControl, sn_inSportControl, sn_otherControl]
    
    sn_incList = []
    for cat in sn_categories:
        sn_incList.append(cat[0])
    sn_decList = []
    for cat in sn_categories:
        sn_decList.append(cat[1])
    sn_nullList = []
    for cat in sn_categories:
        sn_nullList.append(cat[2])
    
    #VAN
    van_mild = queryStats(stats, weight, "van", "mild")
    van_mildMod = queryStats(stats, weight, "van", "m/mod")
    van_mod = queryStats(stats, weight, "van", "moderate")
    van_modSev = queryStats(stats, weight, "van", "mod/sev")
    van_sev = queryStats(stats, weight, "van", "severe")
    van_mixSev = queryStats(stats, weight, "van", "mix severity")
    van_noSev = queryStats(stats, weight, "van", "No Severity")
    van_child = queryStats(stats, weight, "van", "child")
    van_adolescent = queryStats(stats, weight, "van", "adolescent")
    van_adult = queryStats(stats, weight, "van", "adult")
    van_mixAge = queryStats(stats, weight, "van", "mix age")
    van_noAge = queryStats(stats, weight, "van", "No Age")
    van_acute = queryStats(stats, weight, "van", "Acute")
    van_acuteSub = queryStats(stats, weight, "van", "ac/subac")
    van_subacute = queryStats(stats, weight, "van", "subacute")
    van_subChron = queryStats(stats, weight, "van", "subac/chron")
    van_chronic = queryStats(stats, weight, "van", "chronic")
    van_repSub = queryStats(stats, weight, "van", "repsub")
    van_mixChron = queryStats(stats, weight, "van", "mix cnicity")
    van_noChron = queryStats(stats, weight, "van", "No Cnicity")
    van_sport = queryStats(stats, weight, "van", "sport")
    van_military = queryStats(stats, weight, "van", "military")
    van_civilian = queryStats(stats, weight, "van", "civilian")
    van_mixType = queryStats(stats, weight, "van", "mix Type")
    van_healthyControl = queryStats(stats, weight, "van", "HC")
    van_noContactControl = queryStats(stats, weight, "van", "NCC")
    van_inSportControl = queryStats(stats, weight, "van", "ISC")
    van_otherControl = queryStats(stats, weight, "van", "Other Control")
    
    van_categories = [van, van_mild, van_mildMod, van_mod, van_modSev, van_sev, 
                  van_mixSev, van_noSev, van_child, van_adolescent, van_adult,
                 van_mixAge, van_noAge, van_acute, van_acuteSub, van_subacute, van_subChron, van_chronic, van_repSub, 
                 van_mixChron, van_noChron, van_sport, van_military, van_civilian, van_mixType, 
                 van_healthyControl, van_noContactControl, van_inSportControl, van_otherControl]
    
    van_incList = []
    for cat in van_categories:
        van_incList.append(cat[0])
    van_decList = []
    for cat in van_categories:
        van_decList.append(cat[1])
    van_nullList = []
    for cat in van_categories:
        van_nullList.append(cat[2])
        
    #SN/VAN
    snvan_incList = [a + b for a, b in zip(sn_incList, van_incList)]
    snvan_decList = [a + b for a, b in zip(sn_decList, van_decList)]
    snvan_nullList = [a + b for a, b in zip(sn_nullList, van_nullList)]
    
    data_snvan = {'Increase': snvan_incList, 
                'Decrease': snvan_decList, 
                'Null': snvan_nullList}        
    df_snvan = pd.DataFrame(data_snvan, index = detailed_index)
    display(df_snvan)
    
    #DAN------------------------------:
    print("DAN in Detail (" + weight +  "):")
    
    dan_mild = queryStats(stats, weight, "dan", "mild")
    dan_mildMod = queryStats(stats, weight, "dan", "m/mod")
    dan_mod = queryStats(stats, weight, "dan", "moderate")
    dan_modSev = queryStats(stats, weight, "dan", "mod/sev")
    dan_sev = queryStats(stats, weight, "dan", "severe")
    dan_mixSev = queryStats(stats, weight, "dan", "mix severity")
    dan_noSev = queryStats(stats, weight, "dan", "No Severity")
    dan_child = queryStats(stats, weight, "dan", "child")
    dan_adolescent = queryStats(stats, weight, "dan", "adolescent")
    dan_adult = queryStats(stats, weight, "dan", "adult")
    dan_mixAge = queryStats(stats, weight, "dan", "mix age")
    dan_noAge = queryStats(stats, weight, "dan", "No Age")
    dan_acute = queryStats(stats, weight, "dan", "Acute")
    dan_acuteSub = queryStats(stats, weight, "dan", "ac/subac")
    dan_subacute = queryStats(stats, weight, "dan", "subacute")
    dan_subChron = queryStats(stats, weight, "dan", "subac/chron")
    dan_chronic = queryStats(stats, weight, "dan", "chronic")
    dan_repSub = queryStats(stats, weight, "dan", "repsub")
    dan_mixChron = queryStats(stats, weight, "dan", "mix cnicity")
    dan_noChron = queryStats(stats, weight, "dan", "No Cnicity")
    dan_sport = queryStats(stats, weight, "dan", "sport")
    dan_military = queryStats(stats, weight, "dan", "military")
    dan_civilian = queryStats(stats, weight, "dan", "civilian")
    dan_mixType = queryStats(stats, weight, "dan", "mix Type")
    dan_healthyControl = queryStats(stats, weight, "dan", "HC")
    dan_noContactControl = queryStats(stats, weight, "dan", "NCC")
    dan_inSportControl = queryStats(stats, weight, "dan", "ISC")
    dan_otherControl = queryStats(stats, weight, "dan", "Other Control")
    
    dan_categories = [dan, dan_mild, dan_mildMod, dan_mod, dan_modSev, dan_sev, 
                  dan_mixSev, dan_noSev, dan_child, dan_adolescent, dan_adult,
                 dan_mixAge, dan_noAge, dan_acute, dan_acuteSub, dan_subacute, dan_subChron, dan_chronic, dan_repSub, 
                 dan_mixChron, dan_noChron, dan_sport, dan_military, dan_civilian, dan_mixType, 
                 dan_healthyControl, dan_noContactControl, dan_inSportControl, dan_otherControl]
    
    dan_incList = []
    for cat in dan_categories:
        dan_incList.append(cat[0])
    dan_decList = []
    for cat in dan_categories:
        dan_decList.append(cat[1])
    dan_nullList = []
    for cat in dan_categories:
        dan_nullList.append(cat[2])
    
    data_dan = {'Increase': dan_incList, 
                'Decrease': dan_decList, 
                'Null': dan_nullList}        
    df_dan = pd.DataFrame(data_dan, index = detailed_index)
    display(df_dan)
    
    #Visual------------------------------:
    print("Visual in Detail (" + weight +  "):")
    
    vis_mild = queryStats(stats, weight, "vis", "mild")
    vis_mildMod = queryStats(stats, weight, "vis", "m/mod")
    vis_mod = queryStats(stats, weight, "vis", "moderate")
    vis_modSev = queryStats(stats, weight, "vis", "mod/sev")
    vis_sev = queryStats(stats, weight, "vis", "severe")
    vis_mixSev = queryStats(stats, weight, "vis", "mix severity")
    vis_noSev = queryStats(stats, weight, "vis", "No Severity")
    vis_child = queryStats(stats, weight, "vis", "child")
    vis_adolescent = queryStats(stats, weight, "vis", "adolescent")
    vis_adult = queryStats(stats, weight, "vis", "adult")
    vis_mixAge = queryStats(stats, weight, "vis", "mix age")
    vis_noAge = queryStats(stats, weight, "vis", "No Age")
    vis_acute = queryStats(stats, weight, "vis", "Acute")
    vis_acuteSub = queryStats(stats, weight, "vis", "ac/subac")
    vis_subacute = queryStats(stats, weight, "vis", "subacute")
    vis_subChron = queryStats(stats, weight, "vis", "subac/chron")
    vis_chronic = queryStats(stats, weight, "vis", "chronic")
    vis_repSub = queryStats(stats, weight, "vis", "repsub")
    vis_mixChron = queryStats(stats, weight, "vis", "mix cnicity")
    vis_noChron = queryStats(stats, weight, "vis", "No Cnicity")
    vis_sport = queryStats(stats, weight, "vis", "sport")
    vis_military = queryStats(stats, weight, "vis", "military")
    vis_civilian = queryStats(stats, weight, "vis", "civilian")
    vis_mixType = queryStats(stats, weight, "vis", "mix Type")
    vis_healthyControl = queryStats(stats, weight, "vis", "HC")
    vis_noContactControl = queryStats(stats, weight, "vis", "NCC")
    vis_inSportControl = queryStats(stats, weight, "vis", "ISC")
    vis_otherControl = queryStats(stats, weight, "vis", "Other Control")
    
    vis_categories = [vis, vis_mild, vis_mildMod, vis_mod, vis_modSev, vis_sev, 
                  vis_mixSev, vis_noSev, vis_child, vis_adolescent, vis_adult,
                 vis_mixAge, vis_noAge, vis_acute, vis_acuteSub, vis_subacute, vis_subChron, vis_chronic, vis_repSub, 
                 vis_mixChron, vis_noChron, vis_sport, vis_military, vis_civilian, vis_mixType, 
                 vis_healthyControl, vis_noContactControl, vis_inSportControl, vis_otherControl]
    
    vis_incList = []
    for cat in vis_categories:
        vis_incList.append(cat[0])
    vis_decList = []
    for cat in vis_categories:
        vis_decList.append(cat[1])
    vis_nullList = []
    for cat in vis_categories:
        vis_nullList.append(cat[2])
    
    data_vis = {'Increase': vis_incList, 
                'Decrease': vis_decList, 
                'Null': vis_nullList}        
    df_vis = pd.DataFrame(data_vis, index = detailed_index)
    display(df_vis)
    
    #SMN------------------------------:
    print("SMN in Detail (" + weight +  "):")
    
    smn_mild = queryStats(stats, weight, "smn", "mild")
    smn_mildMod = queryStats(stats, weight, "smn", "m/mod")
    smn_mod = queryStats(stats, weight, "smn", "moderate")
    smn_modSev = queryStats(stats, weight, "smn", "mod/sev")
    smn_sev = queryStats(stats, weight, "smn", "severe")
    smn_mixSev = queryStats(stats, weight, "smn", "mix severity")
    smn_noSev = queryStats(stats, weight, "smn", "No Severity")
    smn_child = queryStats(stats, weight, "smn", "child") 
    smn_adolescent = queryStats(stats, weight, "smn", "adolescent")
    smn_adult = queryStats(stats, weight, "smn", "adult")
    smn_mixAge = queryStats(stats, weight, "smn", "mix age")
    smn_noAge = queryStats(stats, weight, "smn", "No Age")
    smn_acute = queryStats(stats, weight, "smn", "Acute")
    smn_acuteSub = queryStats(stats, weight, "smn", "ac/subac")
    smn_subacute = queryStats(stats, weight, "smn", "subacute")
    smn_subChron = queryStats(stats, weight, "smn", "subac/chron")
    smn_chronic = queryStats(stats, weight, "smn", "chronic")
    smn_repSub = queryStats(stats, weight, "smn", "repsub")
    smn_mixChron = queryStats(stats, weight, "smn", "mix cnicity")
    smn_noChron = queryStats(stats, weight, "smn", "No Cnicity")
    smn_sport = queryStats(stats, weight, "smn", "sport")
    smn_military = queryStats(stats, weight, "smn", "military")
    smn_civilian = queryStats(stats, weight, "smn", "civilian")
    smn_mixType = queryStats(stats, weight, "smn", "mix Type")
    smn_healthyControl = queryStats(stats, weight, "smn", "HC")
    smn_noContactControl = queryStats(stats, weight, "smn", "NCC")
    smn_inSportControl = queryStats(stats, weight, "smn", "ISC")
    smn_otherControl = queryStats(stats, weight, "smn", "Other Control")
    
    smn_categories = [smn, smn_mild, smn_mildMod, smn_mod, smn_modSev, smn_sev, 
                  smn_mixSev, smn_noSev, smn_child, smn_adolescent, smn_adult,
                 smn_mixAge, smn_noAge, smn_acute, smn_acuteSub, smn_subacute, smn_subChron, smn_chronic, smn_repSub, 
                 smn_mixChron, smn_noChron, smn_sport, smn_military, smn_civilian, smn_mixType, 
                 smn_healthyControl, smn_noContactControl, smn_inSportControl, smn_otherControl]
    
    smn_incList = []
    for cat in smn_categories:
        smn_incList.append(cat[0])
    smn_decList = []
    for cat in smn_categories:
        smn_decList.append(cat[1])
    smn_nullList = []

    for cat in smn_categories:
        smn_nullList.append(cat[2])
    
    data_smn = {'Increase': smn_incList, 
                'Decrease': smn_decList, 
                'Null': smn_nullList}        
    df_smn = pd.DataFrame(data_smn, index = detailed_index)
    display(df_smn)    
    print("------------------------------\n------------------------------\n------------------------------")
#--end printStats function-----------------------------------------------------------


#RUN IT ON THE DATA!
printStats(stats, "UW")
printStats(stats, "W-TBI")
printStats(stats, "W-Total")

------------------------------

Starting UW Counts...


Network Totals (UW): 


,Increase,Decrease,Null
DMN,23,20,35
ECN,8,7,20
Limbic,3,2,6
Salience/VAN,3,4,11
DAN,1,0,6
Visual,3,3,8
Sensorimotor,4,3,9
Total,45,39,95


All Networks in Detail (UW):


,Increase,Decrease,Null
All,45,39,95
Mild,30,30,55
Mild/Moderate,0,0,10
Moderate,0,0,0
Moderate/Severe,10,3,15
Severe,0,2,1
Mixed Severity,5,4,14
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,4,2,6


Results by Severity & Chronicity (UW)


...

Visual in Detail (W-Total):


,Increase,Decrease,Null
All,197,187,524
Mild,197,187,263
Mild/Moderate,0,0,176
Moderate,0,0,0
Moderate/Severe,0,0,44
Severe,0,0,0
Mixed Severity,0,0,41
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,0,0,0


SMN in Detail (W-Total):


,Increase,Decrease,Null
All,180,193,632
Mild,139,193,370
Mild/Moderate,0,0,176
Moderate,0,0,0
Moderate/Severe,0,0,44
Severe,0,0,0
Mixed Severity,41,0,42
No Severity Stated,0,0,0
Child,0,0,0
Adolescent,0,0,0


------------------------------
------------------------------
------------------------------


In [44]:
#easyQueryStats function--------------------------------------------------------------------------------------------------
#easyQueryStats() works exactly like queryStats except it prints out a mini dataframe 
#of your requested statistics, making it easier to use without additional coding.
#As of 08/11/2021, you also have to provide a weighting type for your results as the second parameter.
#"UW"- unweighted results
#"W-TBI" TBI sample size weighted results
#"W-Total" Total sample size weighted results

def easyQueryStats(stats, weight = "UW", query1="", query2="", query3="", query4="", query5="", query6=""):
    queryList = [query1, query2, query3, query4, query5, query6]
    queryList = [query for query in queryList if not query == ""]
    if(any(query not in nets+results+severities+ages+chronicities+types+controls for query in queryList)):
        print("One of more of your queries did not correspond to a valid keyword. Double check your input!")
        return
    
    if weight not in ["UW", "W-TBI", "W-Total"]:
        print("Your providing weighting choice did not match a valid keyword. Double check your input!")
        return
    
    res = queryStats(stats, weight, query1, query2, query3, query4, query5, query6)
    
    print("You asked for all " + weight + " results corresponding to: " + query1 + " " + query2 + " " + query3 + " " + query4 + " " + query5 + " " + query6)
    print("Here are the results...")
    result = {"Increase": res[0], "Decrease": res[1], "Null": res[2]}
    res_Index = [query1 + " " + query2 + " " + query3 + " " + query4 + " " + query5 + " " + query6]
    df = pd.DataFrame(result, index = res_Index)
    display(df)
    return
#end easyQueryStats function--------------------------------------------------------------------------------------------------    

In [45]:
easyQueryStats(stats, "UW", "limb", "mild", "Acute")
easyQueryStats(stats, "W-TBI", "limb", "mild", "ac/subac")
easyQueryStats(stats, "UW", "limb", "mild", "subacute")
easyQueryStats(stats, "UW", "limb", "mild", "subac/chron")
easyQueryStats(stats, "UW", "limb", "mild", "chronic")

You asked for all UW results corresponding to: limb mild Acute   
Here are the results...


,Increase,Decrease,Null
limb mild Acute,0,0,0


You asked for all W-TBI results corresponding to: limb mild ac/subac   
Here are the results...


,Increase,Decrease,Null
limb mild ac/subac,0,0,47


You asked for all UW results corresponding to: limb mild subacute   
Here are the results...


,Increase,Decrease,Null
limb mild subacute,0,1,2


You asked for all UW results corresponding to: limb mild subac/chron   
Here are the results...


,Increase,Decrease,Null
limb mild subac/chron,1,0,0


You asked for all UW results corresponding to: limb mild chronic   
Here are the results...


,Increase,Decrease,Null
limb mild chronic,1,1,1


In [46]:
#Try it out here!

